In [ ]:
import itertools as it
import numpy as np
from datetime import datetime

def print_board(state):
    print( np.array(state).reshape((3,3)) )

def check(state,val):
    'check if val win in given state or not'
    M = np.array(state).reshape((3,3))
    if  any(sum(x[0])==3*val        for x in (np.rot90(M,k) for k in range(4))):
        return True
    elif any(sum(x[1])==3*val       for x in (np.rot90(M,k) for k in range(2))):
        return True
    elif any(sum(np.diag(x))==3*val for x in (np.rot90(M,k) for k in range(2))):
        return True
    else:
        return False

def win_detector(state):
    'Return -1,0,1 depending upon who is going to win'
    if check(state,1):
        return 1
    if check(state,-1):
        return -1
    return 0

def sym_check(M):
    'checking if a symmetric equivalent entry is already there or not'
    M = M.reshape((3,3))
    if not any(tuple(x.reshape(3*3)) in state_prob for m in (M,M.T) for x in (np.rot90(m,k) for k in range(4))):
        return True
    return False


state_prob = {}
for i in range(10): # 'i' denotes how much entries to be filled
    for j in it.combinations(list(range(9)),i): # 'j' is combination of 'i' entries to filled
        state = np.zeros(3*3,dtype=int)
        for i2 in range(2**i):
            tmp = tuple(map(lambda x:-1+2*int(x),bin(i2)[2:]))
            tmp = (0,)*(i-len(tmp))+tmp
            n1  = tmp.count(1)
            n0  = tmp.count(-1)
            
            if abs(n1-n0)<=1:
                for k in range(i):
                    state[j[k]] = tmp[k]
                if sym_check(state):
                    prob = win_detector(state)
                    state_prob[tuple(state)] = prob if prob==1 else 0

'Total Possible states of game',len(state_prob)

In [ ]:
count = {}
for i in state_prob:
    if 9-i.count(0) in count:
        count[9-i.count(0)]+=1
    else:
        count[9-i.count(0)]=1
count

In [ ]:
def sample_sp(ln):
    'Displaying 10 random states & initial value functions'
    ls_state_prob = list(state_prob)
    np.random.shuffle(ls_state_prob)
    for i in ls_state_prob[:ln]:
        print(np.array(i).reshape((3,3)),state_prob[i])

In [ ]:
A = np.array([0,0,1,0,0,1,0,-1,0,1,-1])
B = np.array([0,0,1,0,0,1,0,-1,-1,1,-1])

def is_next(prv,nxt,val):
    p,n = np.array(prv),np.array(nxt)
    if all( abs(p*n).sum()==x for x in(abs(p).sum(),abs(n).sum()-1) ):
        if sum(p^n)==val:
            return True
    return False

def next_moves(state,val):
    state_vf = {}
    for nxt in state_prob:
        if is_next(state,nxt,val):
            state_vf[nxt] = state_prob[nxt]
    return state_vf

def next_states(state,val):
    "Method will return each next state, with it's value function"
    #Stochastic decision making is to be made by agent
    M = np.array(state).reshape((3,3))
    state_vf = {}
    for M2 in (M,M.T):
        for i in range(4):
            m = np.rot90(M2,i)
            state_vf.update( next_moves(m.reshape(3*3),val) )
    return state_vf

def agent(state,val,stochastic=True):
    nst_vf    = next_states(state,val)
    
    #Choosing next state using probabilities
    ls_nst_vf = list(nst_vf.items())

    #Exploration parameter
    err = 0.25
    
    indxs     = np.array(list(range(len(ls_nst_vf))))
    total_vf  = sum(i[1]+err for i in ls_nst_vf)
    proba     = np.array(list((err+i[1])/total_vf for i in ls_nst_vf))
    
    if stochastic:
        indx  = np.random.choice(indxs,size=1,replace=True,p=proba)[0]
    else:
        indx  = np.argmax(proba)
    
    nxt_state = ls_nst_vf[indx][0]
    
    return nxt_state

In [ ]:
rate,games = 1,300
# Loop for self game based learning
init = datetime.now()
for _ in range(games):
    #Stores the sequence of moves taken in a game
    #print('game {} begin'.format(_+1))
    
    prv_state = (0,)*9
    turn      = -1+2*np.random.randint(2)
    for i in range(9):
        nxt_state = agent(prv_state,turn)
        state_prob[prv_state] = state_prob[prv_state] + rate*(state_prob[nxt_state]-state_prob[prv_state])
        turn = -1+2*(((turn+1)//2)^1)
        prv_state = nxt_state
    #print('game {} ends'.format(_+1))

print('Time taken in training',datetime.now()-init) 

In [ ]:
def practice_game():
    global state_prob,lock
    prv_state = (0,)*9
    turn      = -1+2*np.random.randint(2)
    for i in range(9):
        nxt_state = agent(prv_state,turn)
        lock.acquire()
        state_prob[prv_state] = state_prob[prv_state] + rate*(state_prob[nxt_state]-state_prob[prv_state])
        lock.release()
        turn = -1+2*(((turn+1)//2)^1)
        prv_state = nxt_state

rate,games = 0.05,list(range(10))
init = datetime.now()


import threading                                                                

lock = threading.Lock()

def process(items, start, end):
    for item in items[start:end]:                                               
        practice_game()

def split_processing(items, num_splits=8):                                      
    split_size = len(items) // num_splits                                       
    threads = []                                                                
    for i in range(num_splits):
        start = i * split_size
        end = None if i+1 == num_splits else (i+1) * split_size
        threads.append(                                                         
            threading.Thread(target=process, args=(items, start, end)))         
        threads[-1].start() # start the thread we just created                  

    # wait for all threads to finish                                            
    for t in threads:                                                           
        t.join()

split_processing(games)
print('Time taken in training',datetime.now()-init)

In [ ]:
def play_game():
    print('Apologies for symmetry & tie undeclaring issues')
    game = []
    prv_state = (0,)*9
    turn      = -1+2*np.random.randint(2)
    if turn==-1:
        print('Your chance first')
    elif turn==1:
        print('PC has started game')
    
    for i in range(9):
        if turn==-1:
            print_board(prv_state)
            move = int(input('Enter index ').strip())-1
            while True:
                if not prv_state[move]:
                    break
                else:
                    move = int(input('Enter un-occupied index ').strip())-1
            nxt_state = list(prv_state)
            nxt_state[move] = -1
        elif turn==1:
            nxt_state = agent(prv_state,turn,stochastic=False)
        prv_state = nxt_state
        turn = -1+2*(((turn+1)//2)^1)
        
        win = win_detector(prv_state)
        #print('Winning status',win)
        if win:
            if win==1:
                print('Machine wins............... :p')
                break
            elif win==-1:
                print('Human   wins............... :)')
                break
    else:
        print('Game Tied')
    print_board(prv_state)

In [ ]:
#play_game()

In [ ]:
#sample_sp(1300)